In [1]:
%matplotlib inline

from Basilisk.utilities import (
    SimulationBaseClass,
    macros,
    unitTestSupport,
    simulationArchTypes
)

from Basilisk.simulation import (
    spacecraft,
    extForceTorque,
    simpleNav
)

from Basilisk.fswAlgorithms import (
    inertial3D,
    attTrackingError
)

from Basilisk.architecture import messaging

import numpy as np
import matplotlib.pyplot as plt
import itertools as it
import os

In [2]:
class Simulation:
    def __init__(self, tumble, desired_orientation, sim_max_time, sim_dt, record=True, num_log_points = 150):
        self.sim_task_name = "simTask"
        self.sim_proc_name = "simProc"
        
        self.record = record
        
        self.sim = SimulationBaseClass.SimBaseClass()
        self.sim_max_time = sim_max_time
        self.sim_dt = sim_dt
        
        self.dyn_process = self.sim.CreateNewProcess(self.sim_proc_name, 10)
        self.dyn_process.addTask(self.sim.CreateNewTask(self.sim_task_name, self.sim_dt))
        
        
        # Setup the spaecraft model.
        # The spacecraft model's documentation is found at
        # http://hanspeterschaub.info/basilisk/Documentation/simulation/dynamics/spacecraft/spacecraft.html
        self.spacecraft = spacecraft.Spacecraft()
        self.spacecraft.ModelTag = "bsk-Sat"
        
        # Define the inertial properties
        self.I = [900., 0., 0.,
                  0., 800., 0.,
                  0., 0., 600.]
        
        self.spacecraft.hub.mHub = 750.0  # spacecraft mass [kg]
        self.spacecraft.hub.r_BcB_B = [[0.0], [0.0], [0.0]]  # m - position vector of body-fixed point B relative to CM
        self.spacecraft.hub.IHubPntBc_B = unitTestSupport.np2EigenMatrix3d(self.I)
        self.spacecraft.hub.sigma_BNInit = [[0.1], [0.2], [-0.3]]  # sigma_BN_B
        self.spacecraft.hub.omega_BN_BInit = tumble  # [rad/s]
        
        # Add the spacecraft object to the simulation process
        self.sim.AddModelToTask(self.sim_task_name, self.spacecraft)
        
        # Setup the external control torque
        self.ex_torque = extForceTorque.ExtForceTorque()
        self.ex_torque.ModelTag = "externalDisturbance"
        self.spacecraft.addDynamicEffector(self.ex_torque)
        self.sim.AddModelToTask(self.sim_task_name, self.ex_torque)
        
        # Setup the navigation sensor module which controls the
        # craft's attitude, rate, and position
        self.nav = simpleNav.SimpleNav()
        self.nav.ModelTag = "simpleNavigation"
        self.sim.AddModelToTask(self.sim_task_name, self.nav)
        
        # Setup the inertial 3D guidance module
        self._i3D = inertial3D.inertial3DConfig()
        self.i3D = self.sim.setModelDataWrap(self._i3D)
        self.i3D.ModelTag = "inertial3D"
        self.sim.AddModelToTask(self.sim_task_name, self.i3D, self._i3D)
        self._i3D.sigma_R0N = desired_orientation
        
        # Setup the attitude tracking error evaluation module
        self._attErr = attTrackingError.attTrackingErrorConfig()
        self.attErr = self.sim.setModelDataWrap(self._attErr)
        self.attErr.ModelTag = "attErrorInertial3D"
        self.sim.AddModelToTask(self.sim_task_name, self.attErr, self._attErr)
        
        # Set up recording of values *before* the simulation is initialized
        if self.record:
            t = unitTestSupport.samplingTime(self.sim_max_time, self.sim_dt, num_log_points)
            self.attitude_err_log = self._attErr.attGuidOutMsg.recorder(t)
            self.sim.AddModelToTask(self.sim_task_name, self.attitude_err_log)
        
        # Set up the messaging
        self.nav.scStateInMsg.subscribeTo(self.spacecraft.scStateOutMsg)
        self._attErr.attNavInMsg.subscribeTo(self.nav.attOutMsg)
        self._attErr.attRefInMsg.subscribeTo(self._i3D.attRefOutMsg)
        
    def set_external_torque_cmd_msg(self, msg):
        self.ex_torque.cmdTorqueInMsg.subscribeTo(msg)
        
    def run(self):
        self.sim.InitializeSimulation()
        # self.sim.ConfigureStopTime(self.sim_max_time)
        print(f"Overall stop time is {self.sim_max_time}")
        
        first_leg = self.sim_max_time / 2
        print(f"Simulating up until {first_leg}")
        self.sim.ConfigureStopTime(first_leg)
        self.sim.ExecuteSimulation()
        
        print(f"Simulating now until {self.sim_max_time}")
        self.sim.ConfigureStopTime(self.sim_max_time)
        self.sim.ExecuteSimulation()
        
    def get_plot_data(self):
        if not self.record:
            print("WARNING: Sim did not record!")
            return
        dataLr = self.mrp_log.torqueRequestBody
        dataSigmaBR = self.attitude_err_log.sigma_BR
        dataOmegaBR = self.attitude_err_log.omega_BR_B
        timeAxis = self.attitude_err_log.times()
        
        return dataLr, dataSigmaBR, dataOmegaBR, timeAxis
        
    def plot(dataLr, dataSigmaBR, dataOmegaBR, timeAxis):
        np.set_printoptions(precision=16)

        plt.figure(1)
        for idx in range(3):
            plt.plot(timeAxis * macros.NANO2MIN, dataSigmaBR[:, idx],
                     color=unitTestSupport.getLineColor(idx, 3),
                     label=r'$\sigma_' + str(idx) + '$')
        plt.legend(loc='lower right')
        plt.xlabel('Time [min]')
        plt.ylabel(r'Attitude Error $\sigma_{B/R}$')
        figureList = {}
        pltName = "1"
        figureList[pltName] = plt.figure(1)

        plt.figure(2)
        for idx in range(3):
            plt.plot(timeAxis * macros.NANO2MIN, dataLr[:, idx],
                     color=unitTestSupport.getLineColor(idx, 3),
                     label='$L_{r,' + str(idx) + '}$')
        plt.legend(loc='lower right')
        plt.xlabel('Time [min]')
        plt.ylabel('Control Torque $L_r$ [Nm]')
        pltName = "2" 
        figureList[pltName] = plt.figure(2)

        plt.figure(3)
        for idx in range(3):
            plt.plot(timeAxis * macros.NANO2MIN, dataOmegaBR[:, idx],
                     color=unitTestSupport.getLineColor(idx, 3),
                     label=r'$\omega_{BR,' + str(idx) + '}$')
        plt.legend(loc='lower right')
        plt.xlabel('Time [min]')
        plt.ylabel('Rate Tracking Error [rad/s] ')

In [3]:
# Define Gym environment to train RL-based control of attitude correction
import gymnasium as gym
from ray.rllib.env.env_context import EnvContext

class AttitudeGym(gym.Env):
    metadata = {"render_modes": []}
    
    def __init__(self, config: EnvContext):
        self.action_space = gym.spaces.Box(low=-100, high=100, shape=(3,), dtype=float)
        self.observation_space = gym.spaces.Box(low=-np.inf, high=np.inf, shape=(6,), dtype=float)
        
        self.simulation = None
        self.step_size_ns = config['step_size_ns']
        self.max_mission_time_ns = config['max_mission_time_ns']
        self.run_until_ns = None
        self.action_msg = None
        self.record_sim = config['record_sim']
        self.show_debug=config['show_debug']
        self.iter = None
        
        self.tumble = [[0.8], [-0.6], [0.5]]
        self.desired_ori = [0.0]*3
        
        self.reset()
        
        
    def reset(self, *, seed=None, options=None):
        self.simulation = Simulation(self.tumble, self.desired_ori, self.max_mission_time_ns, self.step_size_ns, record=self.record_sim)
        self.simulation.sim.InitializeSimulation()
        
        self.obs_space_recorder = self.simulation._attErr.attGuidOutMsg.recorder(self.step_size_ns)
        self.simulation.sim.AddModelToTask(self.simulation.sim_task_name, self.obs_space_recorder)
                
        self.run_until_ns = self.step_size_ns
        self.action_msg = messaging.CmdTorqueBodyMsg()
        self.simulation.set_external_torque_cmd_msg(self.action_msg)
        self.iter = 0
        
        self._run()
        
        return self._get_observation(), {}
        
    def step(self, action):
        self._debug_msg(f"Iteration {self.iter}")
        msgData = messaging.CmdTorqueBodyMsgPayload()
        msgData.torqueRequestBody = action
        self._debug_msg(f"Publishing torque request = {action}", tab=True)
        self.action_msg.write(msgData)
        self._run()
        
        obs = self._get_observation()
        self._debug_msg(f"Observation is {obs}", tab=True)
        done = self.run_until_ns >= self.max_mission_time_ns
        reward = self._get_reward()
        self._debug_msg(f"Reward is {reward}", tab=True)
        truncated = done
        
        self.iter += 1
        
        return np.array(obs), reward, done, truncated, {}
        
    def render(self):
        raise NotImplementedError
        
    def close(self):
        pass
        
    def _run(self):
        self.simulation.sim.ConfigureStopTime(self.run_until_ns)
        self.simulation.sim.ExecuteSimulation()
        self.run_until_ns += self.step_size_ns
        
    def _get_observation(self):
        sigma_BR_obs = self.obs_space_recorder.sigma_BR[-1]
        omega_BR_B_obs = self.obs_space_recorder.omega_BR_B[-1]
        
        return list(it.chain.from_iterable([sigma_BR_obs, omega_BR_B_obs]))
    
    def _get_reward(self):
        # Reward the agent for getting the positional data closer to 0
        sigma_BR = self.obs_space_recorder.sigma_BR[-1]
        abs_delta = np.linalg.norm(self.desired_ori - sigma_BR)
        if abs_delta <= 0.1:
            self._debug_msg("Delta between current and desired orientaiton is small! Giving positve reward")
            reward = 10
        else:
            reward = -10.0 * abs_delta
        self._debug_msg(f"sigma_BR = {sigma_BR}, desired orientation is {self.desired_ori}, delta is {self.desired_ori - sigma_BR}, and reward is {reward}", tab=True)
        return reward
    
    def _debug_msg(self, msg, tab=False):
        if self.show_debug:
            if tab:
                msg = '\t' + msg
            print(f"[DEBUG] {msg}")
        

In [ ]:
# Test the basic functionality of the Gym abstraction
import ray

ray.init(ignore_reinit_error=True)
env_config = {
    'step_size_ns': macros.sec2nano(0.1), 
    'max_mission_time_ns': macros.sec2nano(10.0), 
    'record_sim': False, 
    'show_debug': True
}

g = AttitudeGym(env_config)
ray.rllib.utils.check_env(g)
print("AttitudeGym sanity check running...")
for i in it.count():
    obs, reward, done, trunc, info = g.step(g.action_space.sample())
    if done or i > 10:
        print("DONE")
        break
        
ray.shutdown()

2023-03-14 16:52:12,791	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2023-03-14 16:52:13,896	WARNING env.py:156 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.
2023-03-14 16:52:13,896	WARNING env.py:166 -- Your env reset() method appears to take 'seed' or 'return_info' arguments. Note that these are not yet supported in RLlib. Seeding will take place using 'env.seed()' and the info dict will not be returned from reset.
/home/spacetrexlab/.pyenv/versions/basilisk/lib/python3.10/site-packages/gymnasium/spaces/box.py:227: UserWarning: WARN: Casting input x to numpy array.
  logger.warn("Casting input x to numpy array.")


[DEBUG] Iteration 0
[DEBUG] 	Publishing torque request = [-14.60486248  58.24889049  35.07007697]
[DEBUG] 	Observation is [0.12282912313149112, 0.14139647853446594, -0.2941996421491286, 0.7865614386308744, -0.6292689104384549, 0.4837449458071308]
[DEBUG] 	sigma_BR = [ 0.12282912  0.14139648 -0.29419964], desired orientation is [0.0, 0.0, 0.0], delta is [-0.12282912 -0.14139648  0.29419964], and reward is -3.487597841951879
[DEBUG] 	Reward is -3.487597841951879
AttitudeGym sanity check running...
[DEBUG] Iteration 1
[DEBUG] 	Publishing torque request = [-83.45585807  51.65126104  87.00298957]
[DEBUG] 	Observation is [0.13310716266155515, 0.11214292982706298, -0.2907980228993171, 0.7781538871374984, -0.6361449507468878, 0.4813397099383056]
[DEBUG] 	sigma_BR = [ 0.13310716  0.11214293 -0.29079802], desired orientation is [0.0, 0.0, 0.0], delta is [-0.13310716 -0.11214293  0.29079802], and reward is -3.3890565587514034
[DEBUG] 	Reward is -3.3890565587514034
[DEBUG] Iteration 2
[DEBUG] 	Pub

In [5]:
# Define a custom Torch model that just delegates a fully connected net
import ray
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.models.torch.fcnet import FullyConnectedNetwork as TorchFC
from ray.rllib.models import ModelCatalog
from ray.rllib.utils.framework import try_import_torch, try_import_tf

ray.shutdown()
ray.init()

torch, nn = try_import_torch()
tf1, tf, tfv = try_import_tf()

class TorchCustomModel(TorchModelV2, nn.Module):
    """Example of a PyTorch custom model that just delegates to a fc-net."""

    def __init__(self, obs_space, action_space, num_outputs, model_config, name):
        TorchModelV2.__init__(
            self, obs_space, action_space, num_outputs, model_config, name
        )
        nn.Module.__init__(self)

        self.torch_sub_model = TorchFC(
            obs_space, action_space, num_outputs, model_config, name
        )

    def forward(self, input_dict, state, seq_lens):
        input_dict["obs"] = input_dict["obs"].float()
        fc_out, _ = self.torch_sub_model(input_dict, state, seq_lens)
        return fc_out, []

    def value_function(self):
        return torch.reshape(self.torch_sub_model.value_function(), [-1])
    
ModelCatalog.register_custom_model(
    "my_model", TorchCustomModel
)

2023-03-14 16:52:20,536	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


In [10]:
# Define trainer config for Ray training algorithm
from ray.tune.registry import get_trainable_cls

config = (
    get_trainable_cls("PPO")
    .get_default_config()
    # or "corridor" if registered above
    .environment(AttitudeGym, env_config={
        'step_size_ns': macros.sec2nano(0.1), 
        'max_mission_time_ns': macros.sec2nano(10.0), 
        'record_sim': False, 
        'show_debug': False
    })
    .framework("torch")
    .rollouts(num_rollout_workers=os.cpu_count()-1, batch_mode="complete_episodes")
    .training(
        model={
            "custom_model": "my_model",
            "vf_share_layers": True,
        },
        #train_batch_size=10000
    )
    # Use GPUs iff `RLLIB_NUM_GPUS` env var set to > 0.
    #.resources(num_gpus=int(os.environ.get("RLLIB_NUM_GPUS", "0")))
    .resources(num_gpus=1)
)

import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(config.to_dict())

# Define stop conditions
stop_cond = {
    "episode_reward_mean" : 100,  # this is somewhat random
    "training_iteration": 1500
}

{   '_disable_action_flattening': False,
    '_disable_execution_plan_api': True,
    '_disable_preprocessor_api': False,
    '_enable_rl_module_api': False,
    '_enable_rl_trainer_api': False,
    '_fake_gpus': False,
    '_rl_trainer_hps': RLTrainerHPs(),
    '_tf_policy_handles_more_than_one_loss': False,
    'action_space': None,
    'actions_in_input_normalized': False,
    'always_attach_evaluation_results': False,
    'auto_wrap_old_gym_envs': True,
    'batch_mode': 'complete_episodes',
    'callbacks': <class 'ray.rllib.algorithms.callbacks.DefaultCallbacks'>,
    'checkpoint_trainable_policies_only': False,
    'clip_actions': False,
    'clip_param': 0.3,
    'clip_rewards': None,
    'compress_observations': False,
    'create_env_on_driver': False,
    'custom_eval_function': None,
    'custom_resources_per_worker': {},
    'disable_env_checking': False,
    'eager_max_retraces': 20,
    'eager_tracing': False,
    'enable_async_evaluation': False,
    'enable_connectors'

In [ ]:
# Perform the training!
from ray import air, tune
from ray.rllib.utils.test_utils import check_learning_achieved

tuner = tune.Tuner(
    "PPO",
    param_space=config.to_dict(),
    run_config=air.RunConfig(stop=stop_cond)
)

results = tuner.fit()
check_learning_achieved(results, stop_cond['episode_reward_mean'])

(PPO pid=16421) 2023-03-14 16:32:00,677	WARNING algorithm_config.py:596 -- Cannot create PPOConfig from given `config_dict`! Property __stdout_file__ not supported.
(PPO pid=16421) 2023-03-14 16:32:00,857	INFO algorithm.py:506 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(RolloutWorker pid=16505) 2023-03-14 16:32:07,155	WARNING env.py:156 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.
(RolloutWorker pid=16505) 2023-03-14 16:32:07,155	WARNING env.py:166 -- Your env reset() method appears to take 'seed' or 'return_info' arguments. Note that these are not yet supported in RLlib. Seeding will take place using 'env.seed()' and the info dict will not be returned from reset.
(RolloutWorker pid=16505) /home/spacetrexlab/.pyenv/versions/basilisk/lib/python3.10/site-packages/gymnasium/spaces/box.py:227: UserWarning: WARN: Casting inpu

Trial name,agent_timesteps_total,connector_metrics,counters,custom_metrics,date,done,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,episodes_total,experiment_id,hostname,info,iterations_since_restore,node_ip,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_trained,num_env_steps_trained_this_iter,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,pid,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,sampler_results,time_since_restore,time_this_iter_s,time_total_s,timers,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
PPO_AttitudeGym_69621_00000,572000,"{'ObsPreprocessorConnector_ms': 0.007863759994506836, 'StateBufferConnector_ms': 0.0048639774322509766, 'ViewRequirementAgentConnector_ms': 0.1331179141998291}","{'num_env_steps_sampled': 572000, 'num_env_steps_trained': 572000, 'num_agent_steps_sampled': 572000, 'num_agent_steps_trained': 572000}",{},2023-03-14_16-40-53,False,98,{},-475.071,-502.825,-528.549,45,5830,f884378c0f21413b9a27bb8d4e76fa3c,rostration,"{'learner': {'default_policy': {'custom_metrics': {}, 'learner_stats': {'cur_kl_coeff': 1.7085937499999995, 'cur_lr': 5.0000000000000016e-05, 'total_loss': 9.251879592095651, 'policy_loss': 0.012570000792382865, 'vf_loss': 9.225919731201664, 'vf_explained_var': 0.01958377181842763, 'kl': 0.007836763369876008, 'entropy': 5.2169268069728725, 'entropy_coeff': 0.0}, 'model': {}, 'num_grad_updates_lifetime': 132525.5, 'diff_num_grad_updates_vs_sampler_policy': 464.5}}, 'num_env_steps_sampled': 572000, 'num_env_steps_trained': 572000, 'num_agent_steps_sampled': 572000, 'num_agent_steps_trained': 572000}",143,192.168.1.52,572000,572000,572000,4000,572000,4000,0,15,0,0,4000,"{'cpu_util_percent': 28.459999999999997, 'ram_util_percent': 75.0, 'gpu_util_percent0': 0.20400000000000001, 'vram_util_percent0': 0.11789025306940618}",16421,{},{},{},"{'mean_raw_obs_processing_ms': 0.3716388306783318, 'mean_inference_ms': 0.9743967941937052, 'mean_action_processing_ms': 0.19015606776123728, 'mean_env_wait_ms': 1.8058971937511696, 'mean_env_render_ms': 0.0}","{'episode_reward_max': -475.07107021582135, 'episode_reward_min': -528.5492615239958, 'episode_reward_mean': -502.82533281218355, 'episode_len_mean': 98.0, 'episode_media': {}, 'episodes_this_iter': 45, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [-509.46458325784613, -490.00472826675474, -514.207349880867, -484.6124753854084, -512.6711838604322, -500.7591366407024, -511.53008926771133, -523.9118264066032, -496.62787992121406, -521.822154400335, -493.82323796387897, -500.4716139044604, -528.5492615239958, -503.1504142265479, -491.1796628244419, -518.4953066765686, -512.2920778886798, -515.6076020159624, -500.95247766913815, -483.5867239930379, -494.980779015058, -506.64449763764344, -523.0490969684353, -499.52419387244765, -506.7924410447222, -484.2488094478404, -479.3088607606747, -520.0138142240393, -496.42503186258193, -514.2678079391093, -519.5783880313294, -499.0735738915135, -501.27197589531323, -491.87909971717863, -489.6673483792371, -512.8499313125494, -520.4011669127683, -502.0165570897154, -524.00857388096, -509.18061593363376, -508.26550256458137, -488.063975324829, -518.5687123132774, -494.40896270341426, -519.2208278116942, -500.72250276942407, -485.67334916379997, -502.89962448807717, -502.2253076424879, -517.7889551588468, -511.4937134457504, -502.7533544193354, -496.3158479628517, -505.65662502333856, -519.7132827537805, -486.5642258966418, -486.92658761926776, -503.62833854643696, -502.26817696322775, -496.3585046082698, -498.62275350533747, -495.5110274772045, -496.1005355932824, -516.4535256815458, -498.6204434595634, -499.3298449959769, -511.78647204662906, -518.323029732042